<a href="https://colab.research.google.com/github/Henil21/Natural_Language_processing/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in Tensorflow

The main goal of natural language processing (NLP) is to derive information from natural language.

Natural language is a broad term but you can consider it to cover any of the following:

* Text (such as that contained in an email, blog post, book, Tweet)
* Speech (a conversation you have with a doctor, voice commands you give to a smart speaker)


> Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)


In [ ]:
!nvidia-smi  -L

GPU 0: Tesla T4 (UUID: GPU-4a7c27e2-beed-f21f-bdb1-376734ef63b7)


## getting helper functions 🐚

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-05-07 16:29:09--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-05-07 16:29:09 (54.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# importing series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves,compare_historys

## Get a text dataset

>description of data set: text sample of tweet labelled as disaster or not disaster.

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')

--2023-05-07 16:29:15--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.152.128, 142.250.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-05-07 16:29:15 (116 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing Our Data


In [ ]:
import pandas as pd
train_dir=pd.read_csv("train.csv")
test_dir=pd.read_csv("test.csv")
train_dir.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# shuffling the training data
train_shf=train_dir.sample(frac=1,random_state=42)
train_shf.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_dir.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# lets visualize some random training example
import random
random_index=random.randint(0, len(train_dir)-5)
# create random index not higher than total number of samples
for row in train_shf[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"target:{target}","(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("----\n")

target:1 (real disaster)
Text:
70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/1erd2FPryP

----

target:0 (not real disaster)
Text:
Someone walk with me to DQ ??
I wanna Butterfinger Blizzard so bad??

----

target:0 (not real disaster)
Text:
HELLFIRE EP - SILENTMIND &amp; @_bookofdaniel  https://t.co/FKqJY3EzyG

----

target:1 (real disaster)
Text:
Rainstorm Update: Further North or Further South? http://t.co/50vdQ7A1M5 http://t.co/QH6oXfT9Ir

----

target:0 (not real disaster)
Text:
#BakeOffFriends #GBBO 'The one with the mudslide and the guy with the hat'

----



### Split data into training and validation sets ✅

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# use train_test_split to split training data into training and validation sets
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_shf["text"].to_numpy(),
                                                                       train_shf["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [ ]:
val_sentences[:10]
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [ ]:
val_sentences[:10]

array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
       'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
       'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
       '@camilacabello97 Internally and externally screaming',
       'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
       'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
       'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
       "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over southern C

### Converting Text into number ⚡




* Tokenization - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
1. Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.
2. Character-level tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.
3. Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.
* Embeddings - An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use     embeddings:
1. Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.
2. Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

### Text vectorization (Tokenization)
* The TextVectorization layer takes the following parameters:

*  max_tokens - The maximum number of words in your vocabulary (e.g. 20000 or the number of unique words in your text), includes a value for OOV (out of vocabulary) tokens.
* standardize - Method for standardizing text. Default is "lower_and_strip_punctuation" which lowers text and removes all punctuation marks.
* split - How to split text, default is "whitespace" which splits on spaces.
* ngrams - How many words to contain per token split, for example, ngrams=2 splits tokens into continuous sequences of 2.
* output_mode - How to output tokens, can be "int" (integer mapping), "binary" (one-hot encoding), "count" or "tf-idf". See documentation for more.
* output_sequence_length - Length of tokenized sequence to output. For example, if output_sequence_length=150, all tokenized sequences will be 150 tokens long.
* pad_to_max_tokens - Defaults to False, if True, the output feature axis will be padded to max_tokens even if the number of unique tokens in the vocabulary is less than max_tokens. Only valid in certain modes, see docs for more.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#using default textvectorization parameters
text_vec=TextVectorization(max_tokens=None, #how many different words are in our vocabaulary (automatically add <ODV>)
                           standardize="lower_and_strip_punctuation",#convert all to lower case and remove punctucations as it doesnot contribute much in output 
                           split="whitespace",
                           ngrams=None, #create groupe of n-word,
                           output_mode="int",
                           output_sequence_length=None,#how long we want our sequences to be(how long a tweet can be)
                          #  pad_to_max_tokens=True [not valid if max_token is set to None]
                           )


In [ ]:
len(train_sentences[0].split())

7

In [ ]:
# find the average number of token(words) in the training tweet
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                     split="whitespace",
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

# create a smple sentence and tokenize it
sample_tweet="There's a flood in my street"
text_vectorizer([sample_tweet])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# choose a random sentence from training dataset and tokenizing the,
random_sentence=random.choice(train_sentences)
print(f"original text:\n{random_sentence}\n\n vectorized text",text_vectorizer(random_sentence))

original text:
After a violent afternoon storm more severe weather heads for Chicago tonight. Details ----&gt; http://t.co/6Peeip4y7W

 vectorized text tf.Tensor(
[  43    3  357 1721   84   51  272  229 2096   10 1424  383 2495  741
    1], shape=(15,), dtype=int64)


In [ ]:
# Get the unique words in vocabulary
words_in_voc=text_vectorizer.get_vocabulary()
top_5=words_in_voc[:5]
bottom_5=words_in_voc[-5:]
print(f"number of word in vocab {len(words_in_voc)}\n")
print(f"5 most comman word in  vocab {top_5}\n")
print(f"5 least comman  word in vocab {bottom_5}")


number of word in vocab 10000

5 most comman word in  vocab ['', '[UNK]', 'the', 'a', 'in']

5 least comman  word in vocab ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding Layer

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The Parameters we care most about are
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, eg:- a value of 100 would mean each token get represented by a vector of 100 long
* `input_length` = The length of the sequences being passed to the embedding layer

In [ ]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,#set the input shape
                             output_dim=128,
                             input_length=max_length)

In [ ]:
# Get random sentence
random_sentence=random.choice(train_sentences)
print(f"orignal text:\n{random_sentence}")

# embed the random sentence (turn it into dense vector of the fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))



orignal text:
Salvation Army hosts rally to reconnect fathers with children: The Salvation Army is hosting a back to school rallyÛ_ http://t.co/rDjpor3AZg


In [ ]:
#  check out single token embedding
sample_embed[0][0],sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.0446749 , -0.04057628, -0.01240939,  0.01739873, -0.03687781,
         0.03493551,  0.03528431, -0.0415516 ,  0.01274155,  0.04277835,
        -0.03131666,  0.01182238,  0.04673585, -0.02285081,  0.0123657 ,
        -0.04188626,  0.02435331,  0.00783161,  0.0196651 , -0.0461089 ,
        -0.0360099 , -0.00897216,  0.02144365,  0.03952542,  0.0247239 ,
         0.04894635,  0.02404788, -0.03733573, -0.01480088,  0.03881595,
        -0.02796236,  0.0186591 , -0.00867822,  0.00380363,  0.0487071 ,
        -0.02067112, -0.0440392 ,  0.00216711,  0.03284191,  0.0198997 ,
        -0.04584963, -0.00975828, -0.00714908, -0.03814639, -0.02075733,
        -0.03297029, -0.0287848 ,  0.03268846, -0.01765965,  0.02751484,
        -0.02466332, -0.04856155,  0.00498266,  0.02011186, -0.00489664,
        -0.01208273, -0.03520218,  0.04837526,  0.04283113, -0.00968809,
         0.01162722, -0.04930142, -0.04667446,  0.01493445, -0.01635835,
  

### Modelling a text dataset

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data




### Model 0: Getting a baseline
As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert our words to numbers and then model them with the Multinomial Naive Bayes algorithm. 

In [ ]:
# Convert text into number
from sklearn.feature_extraction.text import TfidfVectorizer
# our model
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),# convert words to number using tfidf
    ("clf",MultinomialNB())# model the text
])
# fit the pipleine to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_score=model_0.score(val_sentences,val_labels)
# as we use .evaluate in tf for sklearn its .score
baseline_score*100

79.26509186351706

In [ ]:
baseline_pred=model_0.predict(val_sentences)

baseline_pred[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
train_labels[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

### Creating an evaluation function for our model experiments

we could evaluate these as they are but since we're going to be evaluating several models in the same way going forward, let's create a helper function which takes an array of predictions and ground truth labels and computes the following:

* Accuracy
* Precision
* Recall
* F1-score
> 🔑 Note: Since we're dealing with a classification problem, the above metrics are the most appropriate. If we were working with a regression problem, other metrics such as MAE (mean absolute error) would be a better choice.

In [ ]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def calculate_results(y_true,y_pred):
  # Calculate model accuracy
  model_accuracy=accuracy_score(y_true,y_pred)*100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision":model_precision*100,
                   "recall":model_recall*100,
                   "f1":model_f1*100
                   }
  return model_results

In [ ]:
bline=calculate_results(y_true=val_labels, y_pred=baseline_pred)
bline

{'accuracy': 79.26509186351706,
 'precision': 81.11390004213173,
 'recall': 79.26509186351706,
 'f1': 78.6218975804955}

### Model 1: A Simple Dense Model 🚀

In [ ]:
#  Creating tensorboard callback
from helper_functions import create_tensorboard_callback
SAVE_DIR="model_logs"

In [ ]:
from tensorflow.keras import layers
input=layers.Input(shape=(1,),dtype=tf.string) # inputs are 1-dimensional
x=text_vectorizer(input) # convert strings into number 
x=embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
output=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(input,output,name="model_1_dense")

In [ ]:
# model_1.summary()
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230507-162920
Epoch 1/5
215/215 [==============================] - 27s 90ms/step - loss: 0.6098 - accuracy: 0.6930 - val_loss: 0.5369 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4403 - accuracy: 0.8164 - val_loss: 0.4693 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3457 - accuracy: 0.8610 - val_loss: 0.4572 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2843 - accuracy: 0.8899 - val_loss: 0.4665 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2370 - accuracy: 0.9115 - val_loss: 0.4800 - val_accuracy: 0.7848



> use `GlobalAveragePooling1D` to layer before output layer if our data is 1D

In [ ]:
model_1_pred=probs=model_1.predict(val_sentences)
model_1_pred.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [ ]:
model_1_pred[:10]

array([[0.36844596],
       [0.7556436 ],
       [0.9974969 ],
       [0.11594842],
       [0.08837517],
       [0.933863  ],
       [0.8998223 ],
       [0.99337924],
       [0.9615011 ],
       [0.2934877 ]], dtype=float32)

In [ ]:
tf.squeeze(tf.round(model_1_pred[:10]))==val_labels[:10]

<tf.Tensor: shape=(10,), dtype=bool, numpy=
array([ True, False,  True, False, False,  True,  True,  True,  True,
        True])>

In [ ]:
words_in_voc = text_vectorizer.get_vocabulary()
len(words_in_voc),words_in_voc[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
#  Get weight matrix of embedding layer
embed_weight=model_1.get_layer("embedding").get_weights()[0]
embed_weight

array([[-0.00257788, -0.00067441, -0.02239144, ...,  0.04228447,
         0.02287367,  0.00742683],
       [-0.00587433, -0.03687649, -0.0493554 , ...,  0.02003003,
        -0.01104028,  0.02986543],
       [-0.02821171, -0.01904216,  0.01416913, ...,  0.03740098,
         0.05573111, -0.03430087],
       ...,
       [ 0.03044374, -0.04529284, -0.04590565, ..., -0.00586065,
        -0.00645094, -0.04605695],
       [-0.06411184,  0.03376206, -0.0773836 , ..., -0.07480291,
         0.05802172, -0.00536752],
       [-0.09664132,  0.1081278 , -0.08619222, ..., -0.0875023 ,
         0.05111773, -0.04251654]], dtype=float32)

In [ ]:
# should be same size as vocab size and embedding dim
print(embed_weight.shape)


(10000, 128)


`projector.tensorflow.org`
[word_embeddings guid](https://www.tensorflow.org/text/guide/word_embeddings)
* lets see how we can visualize embedding matrix token representationbn

In [ ]:
#  import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_voc):
#   if num == 0: 
#      continue # skip padding token
#   vec = embed_weight[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## *Recurrent Neural Network (RNN's)⛹*


* RNN's are the useful for sequence data.


* The premise of recurrent neueal network is to use the representation of previous input aid the representation of a later input.🔗


- [MIT Deep Learning Lecture on Recurrent Neural Networks](https://www.youtube.com/watch?v=SEnXr6v2ifU) - explains the background of recurrent neural networks and introduces LSTMs.

- [Understanding LSTMs by Chris Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.


### Model 2 : LSTM
> LSTM = Long short term memory

* out structure of an RNN typically looks like this ⬇

```
Input (text) -> Tokenize -> embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [90]:
# LSTM model
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)

# print(x.shape)
x=layers.LSTM(64,return_sequences=True)(x)
# print(x.shape)

# print(x.shape) 
x=layers.LSTM(64)(x)

outputs=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(input,output,name="model_2_LSTM")

In [53]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
No

In [94]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [95]:
history=model_2.fit(train_sentences,
                    train_labels,
                    epochs=5,
                    validation_data=(val_sentences,val_labels),
                     callbacks=[create_tensorboard_callback(SAVE_DIR,'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230507-185301
Epoch 1/5
215/215 [==============================] - 14s 58ms/step - loss: 0.0349 - accuracy: 0.9848 - val_loss: 1.7979 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0338 - accuracy: 0.9853 - val_loss: 1.7986 - val_accuracy: 0.7559
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0330 - accuracy: 0.9848 - val_loss: 1.7953 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0329 - accuracy: 0.9861 - val_loss: 1.7874 - val_accuracy: 0.7507
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0325 - accuracy: 0.9857 - val_loss: 1.7919 - val_accuracy: 0.7480


In [64]:
model_2_pred_prob=model_2.predict(val_sentences)
model_2_pred_prob[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.52837205],
       [0.8222668 ],
       [0.9997646 ],
       [0.09048042],
       [0.00769633],
       [0.98879087],
       [0.916943  ],
       [0.99994767],
       [0.9997093 ],
       [0.61641175]], dtype=float32)

In [65]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_prob))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [63]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [67]:
model_2_result=calculate_results(y_true=val_labels,
                                 y_pred=model_2_preds)
model_2_result

{'accuracy': 77.55905511811024,
 'precision': 77.5710931680431,
 'recall': 77.55905511811024,
 'f1': 77.45089775603932}